# DS106-03-11-ML - Lesson 3 Hands-On
Decision Trees and Random Forests

---

# Part 1
- use `Titanic` dataset from `seaborn`
- to predict survival
---
- recode or remove string data
- remove missing data
- drop redundant variables
- create a decision tree model
---
- interpret confusion matrix
- intrepret classification report

---

## Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

---
## Load in Data

In [2]:
Titanic = sns.load_dataset('titanic')

---
## View Data

In [3]:
Titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


---
## Data Wrangling
Decision trees and random forests cannot have any string or missing data;
- Recode string variables 
- use the `.dropna` function for missing values

---
## Data Dictionary
[[link to information on dataset]](https://www.kaggle.com/c/titanic/data)


| **Variable** | **Definition** | **Key** |
| --- | --- | --- |
| survival | Survival | 0 = No, 1 = Yes |
| pclass | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd |
| sex | Sex |  |
| Age | Age in years |  |
| sibsp | # of siblings / spouses aboard the Titanic |  |
| parch | # of parents / children aboard the Titanic |  |
| ticket | Ticket number |  |
| fare | Passenger fare |  |
| cabin | Cabin number |  |
| embarked | Port of Embarkation | C = Cherbourg, Q = Queenstown, S = Southampton |

### What kind of data do we have?

In [4]:
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


### Drop the missing data

In [5]:
Titanic.dropna(inplace = True)
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 1 to 889
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     182 non-null    int64   
 1   pclass       182 non-null    int64   
 2   sex          182 non-null    object  
 3   age          182 non-null    float64 
 4   sibsp        182 non-null    int64   
 5   parch        182 non-null    int64   
 6   fare         182 non-null    float64 
 7   embarked     182 non-null    object  
 8   class        182 non-null    category
 9   who          182 non-null    object  
 10  adult_male   182 non-null    bool    
 11  deck         182 non-null    category
 12  embark_town  182 non-null    object  
 13  alive        182 non-null    object  
 14  alone        182 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 18.2+ KB


### Check for redundant variables

In [6]:
Titanic.who.unique()

array(['woman', 'man', 'child'], dtype=object)

In [7]:
Titanic.adult_male.unique()

array([False,  True])

In [8]:
Titanic.pclass.unique()

array([1, 3, 2])

In [9]:
Titanic.survived.value_counts()

1    123
0     59
Name: survived, dtype: int64

In [10]:
Titanic.alive.value_counts()

yes    123
no      59
Name: alive, dtype: int64

### Drop unneeded features (aka variables)
- remove `who` -> this is covered by `sex` and `age`
- remove `adult_male` -> covered by `sex` and `age`
- remove `embark_town` -> same as `embarked`
- remove `class` -> covered by `pclass` and `class` is causing issues in Python (possibly because that is a reserved vocabulary for Python?)
- remove `alive` -> covered by `survived`

In [11]:
Titanic.drop(['who', 'adult_male', 'embark_town', 'class', 'alive', 'alone'], axis = 1, inplace = True)
Titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
1,1,1,female,38.0,1,0,71.2833,C,C
3,1,1,female,35.0,1,0,53.1000,S,C
6,0,1,male,54.0,0,0,51.8625,S,E
10,1,3,female,4.0,1,1,16.7000,S,G
11,1,1,female,58.0,0,0,26.5500,S,C


### Variables must be numeric

In [12]:
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 1 to 889
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   survived  182 non-null    int64   
 1   pclass    182 non-null    int64   
 2   sex       182 non-null    object  
 3   age       182 non-null    float64 
 4   sibsp     182 non-null    int64   
 5   parch     182 non-null    int64   
 6   fare      182 non-null    float64 
 7   embarked  182 non-null    object  
 8   deck      182 non-null    category
dtypes: category(1), float64(2), int64(4), object(2)
memory usage: 13.3+ KB


#### Convert `age` and `fare` to integers

In [13]:
Titanic.age = Titanic.age.astype(int)
Titanic.fare = Titanic.fare.astype(int)

#### Recode `sex`

In [14]:
Titanic.sex.unique()

array(['female', 'male'], dtype=object)

In [15]:
def Sex (series):
    if series == 'female':
        return 0
    if series == 'male':
        return 1
   
Titanic['sexR'] = Titanic['sex'].apply(Sex)
Titanic.drop('sex', axis = 1, inplace = True)
Titanic.sexR = Titanic.sexR.astype(int)

#### Recode `embarked`

In [16]:
Titanic.embarked.value_counts()

S    115
C     65
Q      2
Name: embarked, dtype: int64

In [17]:
def Embarked (series):
    if series == 'S':
        return 0
    if series == 'C':
        return 1
    if series == 'Q':
        return 2
   
Titanic['embarkedR'] = Titanic['embarked'].apply(Embarked)
Titanic.drop('embarked', axis = 1, inplace = True)
Titanic.embarkedR = Titanic.embarkedR.astype(int)

#### Recode `deck`

In [18]:
def Deck (series):
    if series == 'A':
        return 0
    if series == 'B':
        return 1
    if series == 'C':
        return 2
    if series == 'D':
        return 3
    if series == 'E':
        return 4
    if series == 'F':
        return 5
    if series == 'G':
        return 6

Titanic['deckR'] = Titanic['deck'].apply(Deck)
Titanic.drop('deck', axis = 1, inplace = True)
Titanic.deckR = Titanic.deckR.astype(int)

### View Results

In [19]:
Titanic.head()

,survived,pclass,age,sibsp,parch,fare,sexR,embarkedR,deckR
1,1,1,38,1,0,71,0,1,2
3,1,1,35,1,0,53,0,0,2
6,0,1,54,0,0,51,1,0,4
10,1,3,4,1,1,16,0,0,6
11,1,1,58,0,0,26,0,0,2


In [20]:
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 1 to 889
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   survived   182 non-null    int64
 1   pclass     182 non-null    int64
 2   age        182 non-null    int64
 3   sibsp      182 non-null    int64
 4   parch      182 non-null    int64
 5   fare       182 non-null    int64
 6   sexR       182 non-null    int64
 7   embarkedR  182 non-null    int64
 8   deckR      182 non-null    int64
dtypes: int64(9)
memory usage: 14.2 KB


### Define `x` and `y`

In [21]:
# `y` is target, `x` is predictor(s)
x = Titanic.drop('survived', axis=1)
y = Titanic['survived']

---
## Train Test Split

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=76)

---
## Create Initial Decision Tree

In [23]:
decisionTree = DecisionTreeClassifier(random_state=76)
decisionTree.fit(x_train, y_train)

DecisionTreeClassifier(random_state=76)

---
## Assess the Model

In [24]:
treePredictions = decisionTree.predict(x_test)
print(confusion_matrix(y_test, treePredictions))
print(classification_report(y_test, treePredictions))

[[17  2]
 [ 9 27]]
              precision    recall  f1-score   support

           0       0.65      0.89      0.76        19
           1       0.93      0.75      0.83        36

    accuracy                           0.80        55
   macro avg       0.79      0.82      0.79        55
weighted avg       0.84      0.80      0.80        55



---
# Conclusion:
- Model is 84% precise (`weighted average`)
- death prediction: 65% accurate
- survival prediction: 93% accurate

---
---
# Part 2
- create random forest model of `Titanic` dataset to predict survival
- interpret `confusion_matrix`
- interpret `classification_report`

## Create Random Forest Model

In [25]:
forest = RandomForestClassifier(n_estimators=500, random_state=76)
forest.fit(x_train, y_train)

RandomForestClassifier(n_estimators=500, random_state=76)

## Evaluate Model Fit

In [26]:
forestPredictions = forest.predict(x_test)
print(confusion_matrix(y_test, forestPredictions))
print(classification_report(y_test, forestPredictions))

[[14  5]
 [ 7 29]]
              precision    recall  f1-score   support

           0       0.67      0.74      0.70        19
           1       0.85      0.81      0.83        36

    accuracy                           0.78        55
   macro avg       0.76      0.77      0.76        55
weighted avg       0.79      0.78      0.78        55



---
# Conclusion:
- This model is <span style="color:red"> less accurate <span>, at 79% (vs. 84%)(`weighted average`)
- death prediction: <span style="color:green">67% accurate (vs. previous 65%)</span>
- survival prediction: <span style="color:red">85% accurate (vs. previous 93%)</span>
    
#### The Random Forest Model is  worse in overall accuracy, but is better at predicting death specifically.